In [7]:
from flask import Flask, request, jsonify, redirect, render_template
from neo4j import GraphDatabase
import csv

In [8]:
with open("cred.txt") as f1:
    data = csv.reader(f1, delimiter=",")
    for row in data:
        username = row[0]
        password = row[1]
        uri = row[2]
        
print(username, password, uri)

neo4j j4neo bolt://0.0.0.0:7687


In [9]:
# Establish connection
driver = GraphDatabase.driver(uri=uri, auth=(username,password))
session = driver.session()

----

In [15]:
# CREATE STOCK
def create_stock(ticker):
    q1 = """
    create (s:stock{ticker:$ticker})
    """
    map = {"ticker": ticker}
    try:
        session.run(q1, map)
        return(f"stock node is created with ticker name = {ticker}")
    except Exception as e:
        return (str(e))

In [16]:
#create_stock("GME")

'stock node is created with ticker name = GME'

In [17]:
# CREATE SUBREDDIT
def create_subreddit(name):
    q1 = """
    create (sr:subreddit{name:$name})
    """
    map = {"name": name}
    try:
        session.run(q1, map)
        return(f"subreddit node is created with subreddit name = {name}")
    except Exception as e:
        return (str(e))

In [18]:
# CREATE SUBMISSION
def create_submission(subreddit, date, id):
    q1 = '''
    MERGE (sm:submission:'''+subreddit+'''{date:$date, id:$id})
   '''
    map = {"date": date, "id": id}
    try:
        session.run(q1, map)
        return(f"submission node is created with id = {id}")
    except Exception as e:
        return (str(e))

In [19]:
# CREATE COMMENT
def create_comment(subreddit, datetime, comment_id, parent_id):
    q1 = '''
    MERGE (c:comment:'''+subreddit+'''{date:$datetime, comment_id:$comment_id, parent_id:$parent_id})
   '''

    map = {"datetime": datetime, "comment_id": comment_id, "parent_id": parent_id}
    try:
        session.run(q1, map)
        return(f"comment node is created with id = {comment_id}")
    except Exception as e:
        return (str(e))

In [20]:
# CREATE MENTIONS CONNECTION
def create_mentions_connection(node_kind, node_id, ticker):
    
    q1 = """
    MERGE (s:stock{ticker:'"""+ticker+"""'})
    """
    q_submission = """
    MATCH (a:submission),(b:stock)
    WHERE a.id = '"""+node_id+"""' AND b.ticker = '"""+ticker+"""'
    MERGE (a)-[m:MENTIONS]->(b)
    """
    q_comment = """
    MATCH (a:"""+node_kind+"""),(b:stock)
    WHERE a.comment_id = '"""+node_id+"""' AND b.ticker = '"""+ticker+"""'
    MERGE (a)-[m:MENTIONS]->(b)
    """
    map = {"node_kind":node_kind,"node_id": node_id, "ticker":ticker}
    query = q_comment
    if node_kind == 'submission':
        query = q_submission
    try:
        session.run(q1)
        session.run(query, map)
        return(f"connection is created to {ticker}")
    except Exception as e:
        return (str(e))


In [21]:
# CREATE REPLIES CONNECTION
def create_replies_connection(node_kind, reply_from_id, reply_to_id):
    # reply to submission
    q_submission = """
    MATCH 
        (cfrom:comment {comment_id: '"""+reply_from_id+"""'}),
        (cto:submission {id: '"""+reply_to_id+"""'})
    MERGE (cfrom)-[r:REPLIES]->(cto)
    """
    # reply to comment
    q_comment = """
    MATCH 
        (cfrom:comment {comment_id: '"""+reply_from_id+"""'}),
        (cto:comment {comment_id: '"""+reply_to_id+"""'})
    MERGE (cfrom)-[r:REPLIES]->(cto)
    """
    map = {"node_kind":node_kind,"reply_from_id": reply_from_id, "reply_to_id":reply_to_id}
    query = q_comment
    if node_kind == 'submission':
        query = q_submission
    try:
        
        session.run(query, map)
        return(f"connection 'replied' is created")
    except Exception as e:
        return (str(e))

In [12]:
def get_stock(ticker):
    query="""
    "MATCH (s:stock {ticker:$userName}) RETURN s;"
    """
    results = session.run(query)
    data =  results.data()
    return(jsonify(data))

In [13]:
get_stock('GME')

CypherSyntaxError: {code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input '"': expected <init> (line 2, column 1 (offset: 5))
"    "MATCH (s:stock {ticker:$userName}) RETURN s;""
     ^}